### Vectorization and simple neural network in the task of data classification

The aim of the exercise is to acquire skills in the practical use of operations on arrays using the *numpy* library. The content of the task consists of the implementation of a simple neural network used for binary classification (logistic regression).

The task is to fill the code with the necessary operations enabling training of a model based on one neuron with a sigmoidal activation function and a cost function based on binary cross-entropy. The classification itself applies to images depicting birds. At the output of the network (neuron), we expect a binary response interpreted as the probability that the image is a photo of a bird, i.e.: a value close to 1 - there is a bird in the image, a value close to 0 - the image does not depict a bird.

Import of necessary libraries. The PIL library is used to read the file with the test image provided by the user in the last part of the code.

In [ ]:
from ex2_tests import *
import matplotlib.pyplot as plt
import numpy as np
import PIL

%load_ext autoreload
%autoreload 2

Loading the training set from the previously prepared 'train_data.npz' file.

The training set consists of color images with a resolution of $32$
into $32$ pixels stored in an array identified as 'trainx', which has size $m$ by $n$ elements, where $m=2000$ and $n=32∗32∗3=3072$. The images are stored as $n$-element vectors. The images are pre-normalized by dividing by 255.

The second table identified as 'trainy' contains the patterns expected in the network output as '0' - this is not a photo of a bird and '1' - this is a photo of a bird. This array has dimension $m$ on $1$ item.

In [ ]:
data=np.load('train_data.npz')
trainx=np.array(data['trainx'],dtype=np.float64)/255-0.5
trainy=np.array(data['trainy'],dtype=np.float64)
(m,n)=trainx.shape
print('m='+str(m),'n='+str(n))
print(trainx.shape)
print(trainy.shape)

Below we present sample images from the training set with class labels 'bird' and 'non-bird'.

In [ ]:
labels=['non-bird','bird']
fig=plt.figure(figsize=(32,32))
for i in range(0,8):
    fig.add_subplot(1,8,i+1)
    plt.imshow(np.reshape(trainx[i]+0.5,(32,32,3)))
    plt.title(labels[int(trainy[i,0])])
plt.show()

---
**Task 1.**

Implement the sigmoid activation function to work on *numpy* arrays. Use the *np.exp()* function.

In [ ]:
def sigmoid(u):
    v = 1 / (1 + np.exp(-u))
    return v

test_sigmoid(sigmoid)

---
**Task 2.**

In this task, you need to initialize (random values or zeros) an array of weights *w* of dimension $1$ on $n$ elements and the value of the bias weight as a scalar value. 

You can use for example *np.random.normal()* function.

In [ ]:
X=trainx
y=trainy

np.random.seed(42)

w = np.random.normal(size=(1, n))
b = 1

test_init(w,b,X)

---
**Task 3.**

In this task, we want to calculate the value of $u$ for input data $X$, based on the network parameters $w$ and $b$. 

You can use for example $np.dot()$ and $np.transpose()$ functions.

In [ ]:
u = w @ X.T + b

test_u(u,w,b,X)

As the second part of this task calculate $\hat{y}$ based on $u$ and the previously implemented $f(u)$ activation function as *sigmoid()*.

In [ ]:
hy = sigmoid(u)

test_hy(hy,u)

---
**Task 4.**

In this task, you need to prepare an implementation of the cost function *cost(y,hy)*. The function must work on *numpy* arrays. You should use the *np.log()* and *np.sum()* functions. 

Please, **notice** that this function takes two arguments *y* and *hy* as numpy arrays of sizes *(1,m)* (it is only the implementation which should work on any input data). 

You can use for example *np.sum()* and *np.log()* functions.

In [ ]:
def cost(y,hy):
    m = np.shape(y)[1]
    v = (-1) * np.sum(y * np.log(hy) + (1 - y) * np.log(1 - hy)) / m
    return v

test_cost(cost)

**Task 5.**

In the next exercise, our task is to implement the main loop realizing the network training process. The $E$ parameter specifies the number of epochs in the training process. The coefficient $\eta$ (*eta*) defines the length of the learning step. Here, we use a regular *for* loop. Inside the loop, do the following:
- determine the response of the network $\hat{y}$ to the input data $X$,
- calculate corrections for weights in the form of $dw$ and $db$,
- apply weight corrections.

It should be noted that the cost function values are added to the *hist* list in order to later visualize the learning process (error decrease).

In [ ]:
E=12000
eta=0.01
hist=list()
for e in range(0,E):
    u = w @ X.T + b
    hy = sigmoid(u)

    if (e%10==0):
        t=cost(np.transpose(y),hy)
        hist.append(t)

    dw = (1/m) * ((hy - y.T) @ X)
    db = (1/m) * np.sum(hy - y.T)
    w -= eta * dw
    b -= eta * db
    
test_training(w,b,y,X)

Show cost function values for the following epochs.

In [ ]:
plt.plot(hist)
plt.show()

Calculate once more the output of the network for the whole set $X$ of input data.

In [ ]:
u=np.dot(w,np.transpose(X))+b
hy=sigmoid(u)

Evaluate the accuracy of classification for imaga data. Accuracy should be expressed as a percentage.

In [ ]:
v=0
hy=np.round(hy)
for i in range(0,m):
    if (np.transpose(y)[0][i]==hy[0][i]):
        v=v+1
print('Trafnosc klasyfikacji:',100*v/m,'%')

---
Network classification efficiency benchmark for arbitrary test image.

In [ ]:
x=np.array(PIL.Image.open('test3.png'))/255-0.5
plt.imshow(x+0.5)
plt.show()

In [ ]:
T=np.reshape(x,(32*32*3,1))
u=np.dot(w,T)+b
print('hy:',sigmoid(u)[0][0])
t=int(np.round(sigmoid(u)[0][0]))
print('Classification result:',labels[t])

---